# Predict Loan Repayment

We'll be working with financial lending data from [Lending Club](https://www.lendingclub.com/). Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace [here](https://www.lendingclub.com/public/how-peer-lending-works.action).

We will focus on credit modelling that focuses on a borrower's credit risk and define the problem statement for this machine learning project.
- Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

In [14]:
#Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import recall_score, precision_score, f1_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


###Settings###
#inline plotting
%matplotlib inline

#to display more columns
pd.options.display.max_columns = 60

#to auto print variable
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"  

In [15]:
#Load and view data
loans_2007 = pd.read_csv("loans_2007.csv")
print(loans_2007.shape)
loans_2007.head(3)

(42538, 52)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.0,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Data Cleaning
This dataset contains about 42.5k rows and 52 columns. We are working with a preprocessed dataset. Below are the changes that were done.

- removed the desc column
- removed the url column
- removed all columns containing more than 50% missing values

There is still a lot of cleaning and processing that needs to be done.
We shall use the data [dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit) to become familiar with what each column represents and drop columns that we don't need or should not include.

We will look for any features that contain following information.
- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

*"We need to especially pay attention to data leakage, since it can cause our model to overfit."*

After analyzing each column based on the above rules we defined, we can conclude that the following features need to be removed:

- **id**: randomly generated field by Lending Club for unique identification purposes only
- **member_id**: also a randomly generated field by Lending Club for unique identification purposes only
- **funded_amnt**: leaks data from the future (after the loan is already started to be funded)
- **funded_amnt_inv**: also leaks data from the future (after the loan is already started to be funded)
- **grade**: contains redundant information as the interest rate column (int_rate)
- **sub_grade**: also contains redundant information as the interest rate column (int_rate)
- **emp_title**: requires other data and a lot of processing to potentially be useful
- **issue_d**: leaks data from the future (after the loan is already completed funded)
- **zip_code**: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- **out_prncp**: leaks data from the future, (after the loan already started to be paid off)
- **out_prncp_inv**: also leaks data from the future, (after the loan already started to be paid off)
- **total_pymnt**: also leaks data from the future, (after the loan already started to be paid off)
- **total_pymnt_inv**: also leaks data from the future, (after the loan already started to be paid off)
- **total_rec_prncp**: also leaks data from the future, (after the loan already started to be paid off)
- **total_rec_int**: leaks data from the future, (after the loan already started to be paid off),
- **total_rec_late_fee**: also leaks data from the future, (after the loan already started to be paid off),
- **recoveries**: also leaks data from the future, (after the loan already started to be paid off),
- **collection_recovery_fee**: also leaks data from the future, (after the loan already started to be paid off),
- **last_pymnt_d**: also leaks data from the future, (after the loan already started to be paid off),
- **last_pymnt_amnt**: also leaks data from the future, (after the loan already started to be paid off).

Let's first drop these columns.

In [16]:
#drop columns
loans_2007 = loans_2007.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 
                              'emp_title', 'issue_d', 'zip_code', 'out_prncp', 'out_prncp_inv', 
                              'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 
                              'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 
                              'last_pymnt_amnt'], axis=1)

loans_2007.shape

(42538, 32)

We have further reduced the number of columns we are going to use. Now looking for the target column we can see that `loan_status` is the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model.

In [17]:
#understanding target columns
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

There are 8 different possible values for the loan_status column. From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the `Fully Paid` and `Charged Off` values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not.

Let's remove all the loans that don't contain either `Fully Paid` and `Charged Off` as the loan's status and then transform the `Fully Paid` values to `1` for the positive case and the `Charged Off` values to `0` for the negative case.

In [18]:
#filter and replace
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

mapping_dict = {
    "loan_status":{
        "Fully Paid": 1,
        "Charged Off": 0
    }
}
                        
loans_2007 = loans_2007.replace(mapping_dict)

Now we a clean target column and we can consider this to be a `binary classification` problem.

We shall also remove any column with only one unique value as these columns won't be useful for the model since they don't add any information to each loan application.

In [19]:
#drop columns
drop_columns = []

for column in loans_2007.columns:
    unique_non_null = loans_2007[column].dropna().unique()
    if len(unique_non_null) == 1:
        drop_columns.append(column)
        
filtered_loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(filtered_loans_2007.shape)
print(drop_columns)

(38770, 23)
['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


----------------------------
### Feature Engineering

- handling missing values
- converting categorical columns to numeric columns
- removing any other extraneous columns we encounter throughout this process.
- One-hot encoding
- Scale data

In [20]:
#handling missing values
loans = filtered_loans_2007.copy()

null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


`emp_length` this column is important in assessing how risky a potential borrower is?

`title`, `revol_util`, `last_credit_pull_d` all these 3 columms have very low count of na's. so these can be dropped.

`pub_rec_bankruptcies` this column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. We can drop this too.

In [21]:
#drop columns and na rows
loans = loans.drop('pub_rec_bankruptcies', axis=1)

loans = loans.dropna()
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


Let's understand object datatype columns in details.

In [22]:
object_columns_df = loans.select_dtypes(include=['object'])
object_columns_df.head(3)

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016


Studying these columns will tell us that some of the columns seem like they represent categorical values, some columns that represent numeric values and some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful.

We shall clean, drop, transform these columns as applicable referring to data dictionary.

The `home_ownership`, `verification_status`, `emp_length`, and `term` columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

the `purpose` and `title` columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values.

we shall use mapping to clean the `emp_length` column.

the `addr_state` column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [23]:
#clean and transform
loans = loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1)

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.replace(mapping_dict)
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


Now we have a clean dataset to work with. Let's split the data into train and test sets with identical proportions of target classes and perform dummy coding and scale the data.

Note: We will be using RobustScaler for scaling the data to minimize an potential effect from outliers in the data if any.

In [35]:
#split data into train and test
features = loans.loc[:, loans.columns != 'loan_status']
target = loans['loan_status']

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=4, train_size=0.8)

print("Training set with 80% data for training and validation:",X_train.shape)
print("Value counts in Training target column")
print(y_train.value_counts(normalize=True)*100)
print("Test set with 20% data for testing:",X_test.shape)
print("Value counts in Test target column")
print(y_test.value_counts(normalize=True)*100)

Training set with 80% data for training and validation: (30140, 17)
Value counts in Training target column
1    85.660252
0    14.339748
Name: loan_status, dtype: float64
Test set with 20% data for testing: (7535, 17)
Value counts in Test target column
1    85.839416
0    14.160584
Name: loan_status, dtype: float64


In [36]:
#one-hot encoding
categorical_vars = ["home_ownership", "verification_status", "purpose", "term"]
X_train_dummy = pd.get_dummies(X_train[categorical_vars], columns=categorical_vars, drop_first=True)
X_test_dummy = pd.get_dummies(X_test[categorical_vars], columns=categorical_vars, drop_first=True)

print(X_train_dummy.shape)
print(X_test_dummy.shape)

#drop original columns
X_train = X_train.drop(categorical_vars, axis=1)
X_test = X_test.drop(categorical_vars, axis=1)

#convert dummy dataframes to numpy
X_train_dummy = X_train_dummy.to_numpy()
X_test_dummy = X_test_dummy.to_numpy()

(30140, 20)
(7535, 20)


In [42]:
#scale continuous data using RobustScaler
continuous_vars = [col for col in features.columns if col not in categorical_vars]

scaler = RobustScaler()
scaler.fit(X_train[continuous_vars])
X_train_scaled = scaler.transform(X_train[continuous_vars])
X_test_scaled = scaler.transform(X_test[continuous_vars])

#combine scaled and dummy sets
X_train_final = np.concatenate((X_train_scaled, X_train_dummy), axis=1)
X_test_final = np.concatenate((X_test_scaled, X_test_dummy), axis=1)

print(X_train_final.shape)
print(X_test_final.shape)

(30140, 33)
(7535, 33)


These will be our final datasets for training and testing models.

--------------------------
### Model building
We will choose ROC AUC score as our metric for evaluation as there is class imbalance in the target. So we need to be more concerned about `tpr` and `fpr`, i.e. `recall` and `fallout` respectively. We shall also use the `class_weight` parameter to deal with class imbalance.

We shall view this problem from a conservation investor who want to minimize risk of investing in a borrower who may default. This will also decrease the number of options one can invest in potential borrowers. However we shall stick to our case of minimizing risk.

In [43]:
# function to fit model, find best parameters using grid search and cv and print results
def learning_model(model, parameters, train_x, train_y, test_x, test_y, score_metric='roc_auc'):
    #fit model and find best performing parameters
    kf = KFold(10, shuffle=True, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring = score_metric, cv=kf, n_jobs=-1)
    grid_search.fit(train_x, train_y)
    
    best_params = grid_search.best_params_
    train_score = grid_search.best_score_
    test_score = grid_search.score(test_x, test_y)
    predictions = grid_search.predict(test_x)
    
    return best_params, train_score, test_score, predictions

#### Logistic Regression
- **hyperparameter**: *"C"* - trade-off parameter that determines the strength of the regularization
- set *solver: 'lbfgs'* and *max_iter:1000*
- **class_weights**: Yes

In [45]:
#set hyperparameter, fit model and display resutls
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
lr_best_params, lr_train_score, lr_test_score, lr_predictions = learning_model(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), param_grid, X_train_final, y_train, X_test_final, y_test)

tp = len(lr_predictions[(lr_predictions == 1) & (y_test == 1)])
fp = len(lr_predictions[(lr_predictions == 1) & (y_test == 0)])
tn = len(lr_predictions[(lr_predictions == 0) & (y_test == 0)])
fn = len(lr_predictions[(lr_predictions == 0) & (y_test == 1)])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("LogReg Best parameters: {}".format(lr_best_params))
print("LogReg cross-validation roc_auc score: {:.3f}".format(lr_train_score))
print("LogReg Test roc_auc score: {:.3f}".format(lr_test_score))
print("TPR:", tpr)
print("FPR:", fpr)
print("LogReg Confusion Matrix")
pd.crosstab(y_test, lr_predictions,rownames=['True'], colnames=['Predicted'], margins=True)

LogReg Best parameters: {'C': 1}
LogReg cross-validation roc_auc score: 0.707
LogReg Test roc_auc score: 0.695
TPR: 0.6570810142238713
FPR: 0.37207122774133083
LogReg Confusion Matrix


Predicted,0,1,All
True,,,
0,670,397,1067
1,2218,4250,6468
All,2888,4647,7535


We received a `TPR` of 66% and `PFR` of 37%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything.

Let's try to lower the `FPR` with a harsher penality for misclassifying the negative class.
{0: 10, 1: 1}

In [49]:
#set hyperparameter, fit model and display resutls
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
weights = {0: 10, 1: 1}
lr_best_params, lr_train_score, lr_test_score, lr_predictions = learning_model(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight=weights), param_grid, X_train_final, y_train, X_test_final, y_test)

tp = len(lr_predictions[(lr_predictions == 1) & (y_test == 1)])
fp = len(lr_predictions[(lr_predictions == 1) & (y_test == 0)])
tn = len(lr_predictions[(lr_predictions == 0) & (y_test == 0)])
fn = len(lr_predictions[(lr_predictions == 0) & (y_test == 1)])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("LogReg Best parameters: {}".format(lr_best_params))
print("LogReg cross-validation roc_auc score: {:.3f}".format(lr_train_score))
print("LogReg Test roc_auc score: {:.3f}".format(lr_test_score))
print("TPR:", tpr)
print("FPR:", fpr)
print("LogReg Confusion Matrix")
pd.crosstab(y_test, lr_predictions,rownames=['True'], colnames=['Predicted'], margins=True)

LogReg Best parameters: {'C': 0.1}
LogReg cross-validation roc_auc score: 0.706
LogReg Test roc_auc score: 0.695
TPR: 0.39594928880643165
FPR: 0.14526710402999063
LogReg Confusion Matrix


Predicted,0,1,All
True,,,
0,912,155,1067
1,3907,2561,6468
All,4819,2716,7535


We are able lower the false positive rate to 14% from 37% which is great and thus lowered inverstor's risk. But this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. 

### Conclusion
Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

*To be continued with more models...*